In [1]:
import numpy as np
from scipy.linalg import cho_solve, cho_factor
import matplotlib.pyplot as plt
import matplotlib
cmap = 'bwr'

from PIL import Image

In [13]:
import pandas as pd
import re

## load and print data labels
filename_label = "./data/IN15/numorsumarytablenew.txt"
with open(filename_label, 'r') as f:
    labels = f.read().splitlines()

# Split the first line as header, the rest as data
header = [col for col in labels[0].split('\t') if col != 'x/y']
# Remove the empty column (the third column, index 2)
data = [ [entry for entry in row.split('\t') if entry != ''] for row in labels[1:] ]

df_labels = pd.DataFrame(data, columns=header)
print(header)
# print(df_labels[df_labels['numor'].astype(str).str.startswith('3')]['numor'])
data_25C = df_labels[df_labels['SampleName'].str.startswith('Dendrimer') & df_labels['numor'].astype(str).str.startswith('3')]
data_10C = df_labels[df_labels['SampleName'].str.startswith('lt_Dendrimer') & df_labels['numor'].astype(str).str.startswith('3')]
data_40C = df_labels[df_labels['SampleName'].str.startswith('ht_Dendrimer') & df_labels['numor'].astype(str).str.startswith('3')]
data_25C.loc[:, 'SampleName'] = data_25C['SampleName'].str.replace(r'^Dendrimer$', 'Dendrimer_25', regex=True)

# Extract temperature, measurement duration, and wavelength from data_25C

def extract_duration(sample_name):
    match = re.search(r'_(\d+)$', sample_name)
    return int(match.group(1)) if match else None

summary = []
for df, label in zip([data_25C, data_10C, data_40C], ['25C', '10C', '40C']):
    temps = df['TempSa'].unique()
    durations = df['SampleName'].apply(extract_duration).unique()
    wavelengths = df['Lambda'].unique()
    summary.append({
        'Label': label,
        'Temperatures': temps,
        'Durations': durations,
        'Wavelengths': wavelengths
    })

for s in summary:
    print(f"{s['Label']}:")
    print(f"  Temperatures: {s['Temperatures']}")
    print(f"  Durations: {s['Durations']}")
    print(f"  Wavelengths: {s['Wavelengths']}\n")


['SampleName', 'TuneWave', 'numor', 'Average', 'averErr', 'Pol', 'PolErr', 'Echo_khi', 'phmap_stddev', 'phmap_khi', 'MonRate', 'TempSa', 'TempSaErr', 'TempRg', 'TempRgErr', 'Selector', 'Lambda', 'Angle']
25C:
  Temperatures: ['24.99995' '25' '25.00005' '25.000099' '25.000248' '25.000198'
 '24.999901' '24.999851' '24.999802']
  Durations: [25  5 10 50]
  Wavelengths: ['12' '10' '8' '6']

10C:
  Temperatures: ['10.00005' '12.600346' '9.999703' '10.000099' '10.002921' '10.000545'
 '10.001683' '9.9984158' '10.000644' '10.001782' '10.000495' '10.001436'
 '9.9996535' '9.999802' '10.000198' '10.000149']
  Durations: [25  5 10 50]
  Wavelengths: ['10' '12' '6' '8']

40C:
  Temperatures: ['40.000247' '39.996535' '39.999852' '40' '39.999951' '39.999208'
 '39.999753' '39.999406' '40.000049' '40.000198' '39.999752' '40.000099']
  Durations: [ 5 10 25 50]
  Wavelengths: ['10' '12' '6' '8']



In [34]:
duration_list = np.unique([extract_duration(name) for name in data_25C['SampleName']])
conf_list = np.unique(data_25C['Lambda'])
rebin_list = [3, 6, 9]

## 25C

In [45]:
data = data_25C.copy()
path = "./data/IN15/T25/"
rebin = rebin_list[2]

In [46]:
for duration in duration_list:
    Q_list = []
    for conf in conf_list:
        # Find the row in data_25C matching duration and Lambda (conf)
        row = data[(data['SampleName'].str.endswith(f'_{duration}')) & (data['Lambda'] == str(conf))]
        numor = row.iloc[0]['numor']
        filename_e = path+f"sl{rebin}/MD_e_{numor}.txt"
        filename_AP = path+f"sl{rebin}/MD_AP_{numor}.txt"

        # Load the data from filename_AP and extract the Q values and other columns
        # The data file has a header and a line "Tof channel: 0" before the data
        # So skip the first 2 lines (header + "Tof channel: 0")
        data_AP = np.loadtxt(filename_AP, skiprows=2)
        Q = data_AP[:, 0]
        average = data_AP[:, 1]
        averErr = data_AP[:, 2]
        pol = data_AP[:, 3]
        polErr = data_AP[:, 4]

        Q_list.append(Q)

In [47]:
Q_list

[array([0.045665, 0.049401, 0.05291 , 0.056409, 0.059883, 0.063386,
        0.0669  , 0.070395, 0.07402 ]),
 array([0.020299, 0.023592, 0.02645 , 0.029244, 0.032091, 0.034935,
        0.037812, 0.040704, 0.043652]),
 array([0.11217, 0.1183 , 0.12421, 0.13006, 0.13589, 0.14179, 0.14765,
        0.15349, 0.15961]),
 array([0.077346, 0.081959, 0.086387, 0.090768, 0.095134, 0.099556,
        0.10395 , 0.10833 , 0.11291 ])]